In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys 
import os 

sys.path.append('../../../')
from src.athena import Athena
from src.utils import create_session

## Global

In [2]:
boto3_session = create_session(profile_name='dev', role_arn='arn:aws:iam::722696965592:role/athena-full-access-role')

wait = True

sql_path = "../sql/"

athena = Athena(boto3_session=boto3_session, s3_output=os.getenv('ATHENA_S3_OUTPUT'))
athena

Athena(boto3_session=Session(region_name='us-east-1'), s3_output=s3://sql-case-studies/query_results)

## Create Database & Table

In [3]:
response = athena.create_database(
    database='data_bank',
    wait=wait
)

response

Query executed successfully


In [4]:
tables = ['regions', 'customer_nodes', 'customer_transactions']
ddls = {}
for table in tables:
    with open(os.path.join(sql_path, f'{table}_ddl.sql'), 'r') as f:
        ddls[table] = f.read()

    print(ddls[table])
    print('-'*50)
    print('\n')

CREATE EXTERNAL TABLE IF NOT EXISTS data_bank.regions (
  region_id INT,
  region_name VARCHAR(9)
)
COMMENT 'Regions table'
ROW FORMAT DELIMITED
STORED AS PARQUET
LOCATION 's3://sql-case-studies/data_bank/regions/'
TBLPROPERTIES ('classification'='parquet', 'parquet.compress'='SNAPPY');

--------------------------------------------------


CREATE EXTERNAL TABLE IF NOT EXISTS data_bank.customer_nodes (
  customer_id INT,
  region_id INT,
  node_id INT,
  start_date TIMESTAMP,
  end_date TIMESTAMP
)
COMMENT 'Customer nodes table'
ROW FORMAT DELIMITED
STORED AS PARQUET
LOCATION 's3://sql-case-studies/data_bank/customer_nodes/'
TBLPROPERTIES ('classification'='parquet', 'parquet.compress'='SNAPPY');

--------------------------------------------------


CREATE EXTERNAL TABLE IF NOT EXISTS data_bank.customer_transactions (
  customer_id INT,
  txn_date TIMESTAMP,
  txn_type VARCHAR(10),
  txn_amount DOUBLE
)
COMMENT 'Customer transactions table'
ROW FORMAT DELIMITED
STORED AS PARQUET
LOCATIO

In [5]:
for ddl in ddls.values():
    response = athena.create_table( database='data_bank', query=ddl, wait=wait)
    response

Query executed successfully
Query executed successfully
Query executed successfully


## Drop Database & Table

In [5]:
for table in tables:    
    athena.drop_table(
        database='data_bank',
        table=table,
        wait=wait
    )

Query executed successfully
Query executed successfully
Query executed successfully


In [7]:
athena.drop_database(
    database='data_bank',
    wait=wait
)

Query executed successfully
